In [50]:
import nltk
import numpy as np
import pandas as pd 
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import  RidgeClassifier,LogisticRegression
from sklearn.metrics import accuracy_score


# مشروحين بالجزئية الاولى

In [51]:
def removeLinks(text):
    LINK_RS = r'http(s)?\S+'
    PIC_LINK_RS = r'pic\S+'
    result = re.sub(LINK_RS,' ', text)
    result = re.sub(PIC_LINK_RS,' ', result)
    return(result)

# مشروحين بالجزئية الاولى 

In [52]:
 def normalizeLetters(text):
    #لاازالة الحروف المتتكرة في مشكلة انو اذا كانو حربين ولا بعض نفس الشي بيساويهون واحد مثلا https ->htps
    result = re.sub(r'(.)\1+', r'\1', text)
    #تصليح ههه
    #في مشكلة لما تكون هههه باخر الجملة
    result = re.sub(' ه(\s|$)'," ههه " ,result)
    result = re.sub(r"[@,;/(){}\[\]\|]", " ", result)
    result = re.sub("[أآ]", "ا", result)

    return(result)

In [53]:
def hashtags(text): 
    text=re.findall(r"#[\w\d_]+", text)
    return text

In [54]:
def remov_stopwords(sentence):
    stopword = nltk.corpus.stopwords.words('arabic')
    words =[ word for word in  nltk.word_tokenize(sentence) if word not in stopword]
    return " ".join(words)

In [55]:
def remove_hashtags(text):
    text=re.sub(r"#",' ',text)
    text=re.sub(r"(\w+)_(\w+)",' ',text)
    return text

# تحويل الهاش تاغ الى سترينغ عادية 

In [ ]:
def turn_hashtags_to_string(text):
    text=re.sub(r"#|_",r' ',text)
    return text

# معالجة النص 

In [57]:
def data_preprocess(data):
    def sentence_preprocess(sentencs):
        res = removeLinks(sentencs)
        res = normalizeLetters(res)
        res = remov_stopwords(res)
        return res
    array =[sentence_preprocess(sent) for sent in data]
    return array

# قرائة ملفات csv

In [58]:
df_test=pd.read_csv("test.csv",sep="\t")
df_train =pd.read_csv("train_without_any_of_test.csv",sep="\t")

# وضع الداتا في قوائم

In [59]:
index_nan = []
X_train = df_train["tweet"].tolist()
index_nan =[index  for index,val in enumerate(X_train) if str(val) == 'nan'  ]
X_train =[x for x in X_train if str(x) != 'nan']
X_train =data_preprocess(X_train)

y_train = df_train["label"].tolist()
for index in index_nan:
    y_train.pop(index)
y_train =data_preprocess(y_train)


index_nan = []
X_test = df_test["tweet"].tolist()
index_nan =[index  for index,val in enumerate(X_test) if str(val) == 'nan'  ]
X_test =[x for x in X_test if str(x) != 'nan']
X_test =data_preprocess(X_test)


y_test = df_test["label"].tolist()
y_test =[x for x in y_test if str(x) != 'nan']
for index in index_nan:
    y_test.pop(index)
y_test =data_preprocess(y_test)


df_info = pd.DataFrame([["Train",y_train.count("POS"),y_train.count("NEG"),len(y_train)],
                       ["Test",y_test.count("POS"),y_test.count("NEG"),len(y_test)]], columns = ["",'POS', 'NEG',"Count"])
df_info


,,POS,NEG,Count
0,Train,5310,4545,9855
1,Test,3298,3132,6430


# bow من اجل تطبيق خورزميات classfiction 
تحويل النص اللى قيم عددية


In [60]:
def bow_features(X_train, X_test ,min_ngram=1, max_ngram=1):
    
    count_vectorizer = CountVectorizer(ngram_range=(min_ngram, max_ngram))
    X_train_Vec = count_vectorizer.fit_transform(X_train)
    X_test_Vec = count_vectorizer.transform(X_test)
    
    return X_train_Vec, X_test_Vec, count_vectorizer

# tfidf من اجل تطبيق خورزميات classfiction¶
تحويل النص اللى قيم عددية

In [61]:
def tfidf_features(X_train, X_test ,min_ngram=1 ,max_ngram=1):
    
    tfidf_vectorizer = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(min_ngram, max_ngram))
    X_train = tfidf_vectorizer.fit_transform(X_train)
    X_test = tfidf_vectorizer.transform(X_test)
    
    return X_train, X_test, tfidf_vectorizer

تطبيق نوع من خورزميات الماشين لينرنيغ

In [62]:
def train_classifier(X_train, y_train):
    classifier = RidgeClassifier()
    classifier.fit(X_train, y_train)
    return classifier

In [63]:
result={}

تطبيق خورزميات 

In [64]:
def train(X_train,y_train,X_test,y_test,min_ngram,max_ngram):
    tifidf = []
    bag = []
    max_Accuracy_tifidf =0
    max_Accuracy_bag = 0
    for _min in range(min_ngram,max_ngram):
        for _max in range(_min , max_ngram):           
            X_train_bag, X_test_bag, bow_vectorizer = bow_features(X_train, X_test, min_ngram =_min, max_ngram=_max)
            X_train_tfidf, X_test_tfidf, tfidf_vectorizer = tfidf_features(X_train, X_test, min_ngram =_min, max_ngram=_max)
            classifier_bag = train_classifier(X_train_bag, y_train)
            classifier_tfidf = train_classifier(X_train_tfidf, y_train)
            
            y_test_predicted_labels_bag = classifier_bag.predict(X_test_bag)
            y_test_predicted_scores_bag = classifier_bag.decision_function(X_test_bag)
           
            y_test_predicted_labels_tfidf = classifier_tfidf.predict(X_test_tfidf)
            y_test_predicted_scores_tfidf = classifier_tfidf.decision_function(X_test_tfidf)
            
            accuracy_Score_bag = accuracy_score(y_test, y_test_predicted_labels_bag)
            accuracy_Score_tfidf = accuracy_score(y_test, y_test_predicted_labels_tfidf)
            if(accuracy_Score_bag>max_Accuracy_bag):
                max_Accuracy_bag = accuracy_Score_bag
                bag =[bow_vectorizer,classifier_bag]
                
            if(accuracy_Score_tfidf>max_Accuracy_tifidf):
                max_Accuracy_tifidf = accuracy_Score_tfidf
                tifidf =[tfidf_vectorizer,classifier_tfidf]

            
            print(f"min_gram  {_min} , max_gram  {_max}")
            print(f'Bag-of-words Accuracy: {str(accuracy_Score_bag)}')
            print(f'Tfidf Accuracy: {str(accuracy_Score_tfidf)}')
            print("\n\n_____________________________________________________________________________")
            
            
        
    return tifidf,bag


In [65]:
tifidf,bag = train(X_train,y_train,X_test,y_test,1,6)

min_gram  1 , max_gram  1
Bag-of-words Accuracy: 0.7947122861586314
Tfidf Accuracy: 0.8074650077760498


_____________________________________________________________________________
min_gram  1 , max_gram  2
Bag-of-words Accuracy: 0.8082426127527216
Tfidf Accuracy: 0.8132192846034214


_____________________________________________________________________________
min_gram  1 , max_gram  3
Bag-of-words Accuracy: 0.8096423017107309
Tfidf Accuracy: 0.8124416796267496


_____________________________________________________________________________
min_gram  1 , max_gram  4
Bag-of-words Accuracy: 0.8074650077760498
Tfidf Accuracy: 0.8118195956454122


_____________________________________________________________________________
min_gram  1 , max_gram  5
Bag-of-words Accuracy: 0.8034214618973562
Tfidf Accuracy: 0.812597200622084


_____________________________________________________________________________
min_gram  2 , max_gram  2
Bag-of-words Accuracy: 0.7418351477449455
Tfidf Accuracy: 0.

In [66]:
text = input("Enter your tweet: ")
tr_tifidf = tifidf[0].transform([text])
pr_tifidf = tifidf[1].predict(tr_tifidf)

tr_bag = bag[0].transform([text])
pr_bag = bag[1].predict(tr_bag)

print(f"text : {text}\ntifidf : {pr_tifidf[0]}\nbag :{pr_bag[0]}")

Enter your tweet:  اسقاط الولاية 


text : اسقاط الولاية 
tifidf : POS
bag :POS


In [67]:
X_train_reHash = [remove_hashtags(x) for x in X_train]
y_train_reHash = y_train[:]
X_test_reHash = [remove_hashtags(x) for x in X_test]
y_test_reHash =y_test[:]

In [68]:
print("remove hashtags")
tifidf_re,bag_re =train(X_train_reHash,y_train_reHash,X_test_reHash,y_test_reHash,1,6)

remove hashtags
min_gram  1 , max_gram  1
Bag-of-words Accuracy: 0.7762052877138413
Tfidf Accuracy: 0.7914463452566096


_____________________________________________________________________________
min_gram  1 , max_gram  2
Bag-of-words Accuracy: 0.787402799377916
Tfidf Accuracy: 0.7981337480559876


_____________________________________________________________________________
min_gram  1 , max_gram  3
Bag-of-words Accuracy: 0.7869362363919129
Tfidf Accuracy: 0.7964230171073094


_____________________________________________________________________________
min_gram  1 , max_gram  4
Bag-of-words Accuracy: 0.7870917573872472
Tfidf Accuracy: 0.7976671850699845


_____________________________________________________________________________
min_gram  1 , max_gram  5
Bag-of-words Accuracy: 0.7853810264385692
Tfidf Accuracy: 0.7979782270606531


_____________________________________________________________________________
min_gram  2 , max_gram  2
Bag-of-words Accuracy: 0.7398133748055987
Tf

In [69]:
text = input("Enter your tweet: ")
tr_tifidf_re = tifidf_re[0].transform([text])
pr_tifidf_re = tifidf_re[1].predict(tr_tifidf_re)

tr_bag_re = bag_re[0].transform([text])
pr_bag_re = bag_re[1].predict(tr_bag_re)

print(f"text : {text}\ntifidf : {pr_tifidf_re[0]}\nbag :{pr_bag_re[0]}")

Enter your tweet:  اسقاط الولاية


text : اسقاط الولاية
tifidf : POS
bag :POS


In [70]:
X_train_strHash = [turn_hashtags_to_string(x) for x in X_train]
y_train_strHash = y_train[:]
X_test_strHash = [turn_hashtags_to_string(x) for x in X_test]
y_test_strHash =y_test[:]

In [71]:
print("string hashtags")
tifidf_str,bag_str=train(X_train_strHash,y_train_strHash,X_test_strHash,y_test_strHash,1,6)

string hashtags
min_gram  1 , max_gram  1
Bag-of-words Accuracy: 0.8017107309486781
Tfidf Accuracy: 0.8186625194401245


_____________________________________________________________________________
min_gram  1 , max_gram  2
Bag-of-words Accuracy: 0.8192846034214619
Tfidf Accuracy: 0.8265940902021773


_____________________________________________________________________________
min_gram  1 , max_gram  3
Bag-of-words Accuracy: 0.8200622083981337
Tfidf Accuracy: 0.8279937791601866


_____________________________________________________________________________
min_gram  1 , max_gram  4
Bag-of-words Accuracy: 0.8209953343701399
Tfidf Accuracy: 0.8264385692068429


_____________________________________________________________________________
min_gram  1 , max_gram  5
Bag-of-words Accuracy: 0.8203732503888025
Tfidf Accuracy: 0.8262830482115086


_____________________________________________________________________________
min_gram  2 , max_gram  2
Bag-of-words Accuracy: 0.7802488335925349
T

In [72]:
text = input("Enter your tweet: ")
tr_tifidf_str = tifidf_str[0].transform([text])
pr_tifidf_str = tifidf_str[1].predict(tr_tifidf_str)

tr_bag_str = bag_str[0].transform([text])
pr_bag_str = bag_str[1].predict(tr_bag_str)

print(f"text : {text}\ntifidf : {pr_tifidf_str[0]}\nbag :{pr_bag_str[0]}")

Enter your tweet:  سعوديات مع الولاية


text : سعوديات مع الولاية
tifidf : POS
bag :POS


In [73]:
text = input("Enter your tweet: ")
tr_tifidf_str = tifidf_str[0].transform([text])
pr_tifidf_str = tifidf_str[1].predict(tr_tifidf_str)

tr_bag_str = bag_str[0].transform([text])
pr_bag_str = bag_str[1].predict(tr_bag_str)

print(f"text : {text}\ntifidf : {pr_tifidf_str[0]}\nbag :{pr_bag_str[0]}")

Enter your tweet:  اسقاط الولية


text : اسقاط الولية
tifidf : NEG
bag :POS


In [74]:
text = input("Enter your tweet: ")
tr_tifidf_str = tifidf_str[0].transform([text])
pr_tifidf_str = tifidf_str[1].predict(tr_tifidf_str)

tr_bag_str = bag_str[0].transform([text])
pr_bag_str = bag_str[1].predict(tr_bag_str)

print(f"text : {text}\ntifidf : {pr_tifidf_str[0]}\nbag :{pr_bag_str[0]}")

Enter your tweet:  سحر الجنوب


text : سحر الجنوب
tifidf : NEG
bag :POS


In [75]:
text = input("Enter your tweet: ")
tr_tifidf_str = tifidf_str[0].transform([text])
pr_tifidf_str = tifidf_str[1].predict(tr_tifidf_str)

tr_bag_str = bag_str[0].transform([text])
pr_bag_str = bag_str[1].predict(tr_bag_str)

print(f"text : {text}\ntifidf : {pr_tifidf_str[0]}\nbag :{pr_bag_str[0]}")

Enter your tweet:  هذا اللي جانا من ورا اسقاط الولاية مع الأسف


text : هذا اللي جانا من ورا اسقاط الولاية مع الأسف
tifidf : POS
bag :POS


In [76]:
text = input("Enter your tweet: ")
tr_tifidf_str = tifidf_str[0].transform([text])
pr_tifidf_str = tifidf_str[1].predict(tr_tifidf_str)

tr_bag_str = bag_str[0].transform([text])
pr_bag_str = bag_str[1].predict(tr_bag_str)

print(f"text : {text}\ntifidf : {pr_tifidf_str[0]}\nbag :{pr_bag_str[0]}")

Enter your tweet:  سعوديات ضد الولاية 


text : سعوديات ضد الولاية 
tifidf : NEG
bag :POS
